In [2]:
import pandas as pd
import os
import json

from data_utils import DATA_PATH, YEARS

In [8]:
for year in YEARS:
    poll_fpath = os.path.join(DATA_PATH, f'polls-{year}.json')
    with open(poll_fpath) as f:
        polls = json.load(f)
    for poll in polls:
        poll['meta']['year'] = year
        poll['meta']['query'] = 'vote trump biden'


In [7]:
os.path.join(DATA_PATH, f'polls-{year}.json')


{'type': 'twitter',
 'trump_raw': 50,
 'biden_raw': 9,
 'end_datetime': '12/03/2020',
 'num_votes': 59,
 'trump_perc': 84.7457627118644,
 'biden_perc': 15.254237288135593,
 'meta': {'duration_minutes': 10080,
  'id': '1331794489137041412',
  'options': [{'position': 1, 'label': 'For Biden', 'votes': 9},
   {'position': 2, 'label': 'Against Trump', 'votes': 50}],
  'text': 'If you voted the Democrat ticket for president, was it a vote for Biden or against Trump?',
  'author_id': '1292939157375987712',
  'tweet_id': '1331794490382819331',
  'author_name': 'KalamityJohn',
  'year': 2020}}